In [1]:
%load_ext autoreload
%autoreload 2
from confignb import *

INFO:2026-01-25 15:18:06,530:jax._src.xla_bridge:834: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
15:18:06: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [9]:
X, y, pts, ks = h.load_data(DATA_DIR / 'ASCADv_clean.h5')
best_feats_idx = np.load('data/rf_rfe_5_best.npy')
X = X.rename(columns=str)
X_best = X.iloc[:, best_feats_idx[:5]]

In [7]:
from scipy import fft
from sklearn.base import TransformerMixin


def _feat_window(X, f, n, k):
    i = list(n).index(f)
    return X[:, i-k:i+k]

@h.dataclass
class FFTFeaturizer(TransformerMixin):
    feats: list
    names: list
    k: int = 8
    n: int = 8

    def _fft_feat(self, W):
        freqs = fft.rfft(W, axis=1)
        return np.abs(freqs)[:, 1:self.n+1]

    def fit(self, X, y):
        return self

    def transform(self, X):
        X = np.asarray(X, dtype=np.float32)
        windows = [_feat_window(X, f, self.names, self.k)
                   for f in self.feats]
        return np.concat([self._fft_feat(W) for W in windows], axis=1)

In [13]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([
    ("fft", FFTFeaturizer(["187", "1070"], X_best.columns, k=4, n=1)),
])
rf = m.SimpleRFC(max_depth=10)
pl_rf = m.make_pipeline(feats, rf)

h.cv(SEED, pl_rf, X_best, y, pts, ks)

/home/notuser/workspace/SCA/.venv/lib/python3.13/site-packages/sklearn/ensemble/_forest.py:993: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)
15:20:40: [1] Mean PI: -inf
/home/notuser/workspace/SCA/.venv/lib/python3.13/site-packages/sklearn/ensemble/_forest.py:993: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)
15:20:45: [2] Mean PI: -inf


KeyboardInterrupt: 